In [1]:
import numpy as np
import random

# Sampler for loose Self Avoiding Random Walks (SARW)

We define a **loose** SARW as a SARW that is allowed to be stuck, i.e. a **loose** SARW of length $L$ may move for $C$ steps, where $C < L$ before having no possible moves.

In [69]:
# Returns a list of possible next coordinates
def get_candidates(coord, visited):
    candidates = []
    
    for dx, dy in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
        new_coord = (coord[0] + dx, coord[1] + dy)
        if new_coord not in visited:
            candidates.append(new_coord)
    return candidates


# Generates a loose SARW of length L
def generate_loose_sarw(L):
    coords = [(0, 0)]
    visited = set(coords)
    while len(coords) <= L:
        candidates = get_candidates(coords[-1], visited)
        if len(candidates) == 0: # no possible moves, keep still
            coords += [coords[-1]]*(L+1-len(coords))
            break
        coords.append(random.choice(candidates))
        visited.add(coords[-1])
    return coords


In [102]:
print("Generated loose SARW of length 10:")
print(generate_loose_sarw(10))

print("\nGenerated stuck SARW of length 10:")
stuck_example = None
while stuck_example == None:
    curr = generate_loose_sarw(10)
    if len(curr) != len(set(curr)):
        stuck_example = curr
print(stuck_example)


Generated loose SARW of length 10:
[(0, 0), (0, -1), (-1, -1), (-1, -2), (-1, -3), (0, -3), (0, -2), (1, -2), (2, -2), (2, -1), (3, -1)]

Generated stuck SARW of length 10:
[(0, 0), (0, 1), (-1, 1), (-2, 1), (-2, 0), (-2, -1), (-2, -2), (-1, -2), (-1, -1), (-1, 0), (-1, 0)]


# Estimating $C_L$

Using the loose SARW sampler, we can estimate $C_L$ with importance sampling. To do so, we first sample n loose SARW and weight each walk based on its survival probability, i.e. a higher survival probability would give it higher weightage.

$W_i = \prod^L_{t=1} m_t$, where $m_t$ is the number of valid moves at step $t$.

The estimate for $c_L$ is then $c_L \approx \frac{1}{n} \sum^n_{i=1} W_i$

In [126]:
def get_weight(walk):
    visited = set()
    product = 1
    for coord in walk[:-1]:
        visited.add(coord)
        m = len(get_candidates(coord, visited))
        product *= m
        if product == 0: # early termination
            break
    return product

def estimate_C_n(L, n):
    total_weight = 0
    for _ in range(n):
        walk = generate_loose_sarw(L)
        total_weight += get_weight(walk)
    return total_weight / n

In [130]:
estimate_C_n(50, 100_000)

5.35178493558573e+21